# LLM Tutorial

**Objective**: Predict whether a COVID‑19 patient had a sever outcome - an inpatient or emergency encounter - within 30 days after onset of COVID.

**Data Source**: SyntheticMass [COVID-19 10K, CSV](https://mitre.box.com/shared/static/9iglv8kbs1pfi7z8phjl9sbpjk08spze.zip)

## Import Dataset

We'll use three tables: `patients`, `encounters` and `conditions`.

In [ ]:
import pandas as pd